Write a DL program to use retrained model: ResNet to classify the Pet : Cat or Dog

In [3]:
import numpy as np
import tensorflow as tf

In [4]:
# 1. Preprocessing of Data : Image Generators + Train test split
# 2. Model import and deactivate the learnings of layers
# 3. Creation of input layer 
# 4. Customization of imported model by adding Flatten and Output layer
# 5. Create model by combination of Input and Output layers.
# 6. Compile Model
# 7. Train Model. 
# 8. Deployment | Prediction

In [5]:
class MyCLRuleMonitor(tf.keras.callbacks.Callback):
  def __init__(self, CL):
    super(MyCLRuleMonitor).__init__()
    self.CL = CL

  def on_epoch_end(self, epoch, logs=None):
    trainScore = logs["accuracy"]
    testScore = logs["val_accuracy"]

    if testScore > trainScore and testScore >= self.CL:
      self.model.stop_training = True

In [6]:
train_image_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0)
test_image_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale= 1.0/255.0)

In [7]:
# Train and Test data:
train_image_data = train_image_data_generator.flow_from_directory('cats_and_dogs/cats_and_dogs/train', 
                                                                  batch_size = 20,
                                                                  class_mode = 'binary',
                                                                  target_size = (256,256))

test_image_data = test_image_data_generator.flow_from_directory('cats_and_dogs/cats_and_dogs/validation',
                                                                batch_size = 20,
                                                                class_mode= 'binary',
                                                                target_size = (256,256))

Found 2000 images belonging to 2 classes.
Found 1002 images belonging to 2 classes.


In [8]:
resnet = tf.keras.applications.ResNet101(include_top= False)

In [9]:
resnet.summary()

Model: "resnet101"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None,      │          0 │ -                 │
│ (InputLayer)        │ None, 3)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, None,      │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ None, 3)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, None,      │      9,472 │ conv1_pad[0][0]   │
│                     │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, None,      │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, None,      │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, None,      │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, None,      │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, None,      │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, None,      │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, None,      │          0 │ conv2_block1_1_b… │
│ (Activation)        │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, None,      │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, None,      │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, None,      │          0 │ conv2_block1_2_b… │
│ (Activation)        │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, None,      │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ None, 256)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, None,      │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ None, 256)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, None,      │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ None, 256)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, None,      │      1,024 │ conv2_block1_3_c

 Total params: 42,658,176 (162.73 MB)

 Trainable params: 42,552,832 (162.33 MB)

 Non-trainable params: 105,344 (411.50 KB)

In [10]:
for layer in resnet.layers:
    layer.trainable= False

In [11]:
input = tf.keras.layers.Input(shape=[256,256,3])

x = resnet(input)

x1 = tf.keras.layers.Flatten()(x)
x2 = tf.keras.layers.Dense(units = 512, activation= 'relu', name = 'h1')(x1)
x3 = tf.keras.layers.Dense(units = 512, activation= 'relu', name = 'h2')(x2)
x4 = tf.keras.layers.Dense(units = 512, activation= 'relu', name = 'h3')(x3)
x5 = tf.keras.layers.Dense(units = 512, activation= 'relu', name = 'h4')(x4)

output = tf.keras.layers.Dense(units= 1, activation= 'sigmoid', name = 'output')(x5)
model = tf.keras.models.Model(inputs= input, outputs = output)
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet101 (Functional)          │ (None, 8, 8, 2048)     │    42,658,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 131072)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ h1 (Dense)                      │ (None, 512)            │    67,109,376 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ h2 (Dense)                      │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ h3 (Dense)                      │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ h4 (Dense)                      │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 110,556,033 (421.74 MB)

 Trainable params: 67,897,857 (259.01 MB)

 Non-trainable params: 42,658,176 (162.73 MB)

In [12]:
model.compile(optimizer= 'adam',
              loss = 'binary_crossentropy',
              metrics = ['accuracy']
              )

In [13]:
model.fit(train_image_data,
            validation_data = test_image_data,
            steps_per_epoch= (len(train_image_data.filenames) // train_image_data.batch_size),
            validation_steps= (len(test_image_data.filenames) // test_image_data.batch_size),
            epochs= 20,
            callbacks = [MyCLRuleMonitor(0.9)])

Epoch 1/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 865s 8s/step - accuracy: 0.5480 - loss: 1.1990 - val_accuracy: 0.6310 - val_loss: 0.6350
Epoch 2/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 835s 8s/step - accuracy: 0.6125 - loss: 0.6601 - val_accuracy: 0.5380 - val_loss: 0.6638
Epoch 3/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 787s 8s/step - accuracy: 0.6385 - loss: 0.6522 - val_accuracy: 0.6330 - val_loss: 0.6692
Epoch 4/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 816s 8s/step - accuracy: 0.6385 - loss: 0.6443 - val_accuracy: 0.6280 - val_loss: 0.6489
Epoch 5/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 822s 8s/step - accuracy: 0.6750 - loss: 0.6104 - val_accuracy: 0.6950 - val_loss: 0.6171
Epoch 6/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 744s 7s/step - accuracy: 0.6455 - loss: 0.6244 - val_accuracy: 0.6610 - val_loss: 0.6122
Epoch 7/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 29456s 297s/step - accuracy: 0.6665 - loss: 0.6253 - val_accuracy: 0.6750 - val_loss: 0.6101
Epoch 8/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 1144s 11s/step - accuracy: 0.7070 - loss: 0.5825 - va

KeyboardInterrupt: 

In [ ]:
# Input | Deployment

img = tf.keras.preprocessing.image.load_img('catt.jpg', target_size = (256,256))
img_arr = tf.keras.preprocessing.image.img_to_array(img)

img_arr.shape

np_img = np.expand_dims(img_arr, axis = 0)
np_img.shape

Not seeing that good Impreovements in the model, accuracy and val_accuracy is kinda stucked